In [148]:
!python -m venv hw3

In [159]:
!"C:\Users\Nadia Copello\Documents\GitHub\High-School-Access-Peru\hw3\Scripts\python.exe" -m pip install -r "C:\Users\Nadia Copello\Documents\GitHub\High-School-Access-Peru\requirements.txt"


^C


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [2527 lines of output]
  <string>:19: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  C:\Users\Nadia Copello\AppData\Local\Temp\pip-build-env-kaph7kog\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
  !!
  
          ********************************************************************************
          Please consider removing the following classifiers in favor of a SPDX license expression:
  
          License :: OSI Approved :: BSD License
  
          See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
          ********************************************************************************
  
  !!
    self._finalize_license_expression()
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-312\p

     ---------------------------------------- 0.0/5.2 MB ? eta -:--:--
     -------- ------------------------------- 1.0/5.2 MB 5.6 MB/s eta 0:00:01
     -------------- ------------------------- 1.8/5.2 MB 4.6 MB/s eta 0:00:01
     ------------------ --------------------- 2.4/5.2 MB 3.8 MB/s eta 0:00:01
     ---------------------- ----------------- 2.9/5.2 MB 4.0 MB/s eta 0:00:01
     ------------------------ --------------- 3.1/5.2 MB 3.4 MB/s eta 0:00:01
     -------------------------- ------------- 3.4/5.2 MB 3.1 MB/s eta 0:00:01
     ---------------------------- ----------- 3.7/5.2 MB 2.8 MB/s eta 0:00:01
     -------------------------------- ------- 4.2/5.2 MB 2.5 MB/s eta 0:00:01
     ---------------------------------- ----- 4.5/5.2 MB 2.4 MB/s eta 0:00:01
     ---------------------------------- ----- 4.5/5.2 MB 2.4 MB/s eta 0:00:01
     -------------------------------------- - 5.0/5.2 MB 2.1 MB/s eta 0:00:01
     ---------------------------------------- 5.2/5.2 MB 2.1 MB/s eta 0

In [158]:
!"C:\Users\Nadia Copello\Documents\GitHub\High-School-Access-Peru\hw3\Scripts\python" -m pip list

Package Version
------- -------
pip     24.2


In [2]:
pip install contextily


   ---------------------------------------- 0.0/25.4 MB ? eta -:--:--
   - -------------------------------------- 1.0/25.4 MB 5.6 MB/s eta 0:00:05
   -- ------------------------------------- 1.8/25.4 MB 4.6 MB/s eta 0:00:06
   ---- ----------------------------------- 2.9/25.4 MB 5.1 MB/s eta 0:00:05
   ------ --------------------------------- 3.9/25.4 MB 4.9 MB/s eta 0:00:05
   ------ --------------------------------- 4.2/25.4 MB 4.6 MB/s eta 0:00:05
   ------ --------------------------------- 4.2/25.4 MB 4.6 MB/s eta 0:00:05
   ------- -------------------------------- 4.7/25.4 MB 3.4 MB/s eta 0:00:07
   -------- ------------------------------- 5.2/25.4 MB 3.1 MB/s eta 0:00:07
   -------- ------------------------------- 5.5/25.4 MB 3.0 MB/s eta 0:00:07
   --------- ------------------------------ 6.0/25.4 MB 2.8 MB/s eta 0:00:07
   --------- ------------------------------ 6.3/25.4 MB 2.7 MB/s eta 0:00:08
   --------- ------------------------------ 6.3/25.4 MB 2.7 MB/s eta 0:00:08
   --

In [3]:
# Análisis de Escuelas en Perú: Mapas Estáticos y Análisis de Proximidad
# Importamos las bibliotecas necesarias
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon
import contextily as ctx
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import matplotlib.patches as mpatches

In [4]:
# Vemos el archivo como HTML 
archivo_html = 'listado_iiee.xls'  

# pd.read_html 
df = pd.read_html(archivo_html)[0]

#  Verifico
print(df.head())

   Código Modular  Anexo                      Nombre de SS.EE.  Ubigeo  \
0          257048      0               001 NIÑO JESUS DE PRAGA   10101   
1          257055      0            002 RAQUEL ROBLES DE ROMAN   10101   
2          257097      0  006 MARIA PALMIR SANCHEZ DE REATEGÜI   10101   
3          568097      0                                   020   10101   
4          667576      0        025 BRYAN ANTONIO LOPEZ CASTRO   10101   

  Departamento    Provincia     Distrito  Código DRE/UGEL   DRE / UGEL  \
0     AMAZONAS  CHACHAPOYAS  CHACHAPOYAS            10001  CHACHAPOYAS   
1     AMAZONAS  CHACHAPOYAS  CHACHAPOYAS            10001  CHACHAPOYAS   
2     AMAZONAS  CHACHAPOYAS  CHACHAPOYAS            10001  CHACHAPOYAS   
3     AMAZONAS  CHACHAPOYAS  CHACHAPOYAS            10001  CHACHAPOYAS   
4     AMAZONAS  CHACHAPOYAS  CHACHAPOYAS            10001  CHACHAPOYAS   

  Centro Poblado  Código Centro Poblado  Código Local  \
0    CHACHAPOYAS               131197.0          19.0

In [7]:
# Cargamos los datos de distritos de Perú (shapefile)
# Usamos el que nos dieron
peru_distritos = gpd.read_file('C:/Users/Nadia Copello/Documents/GitHub/High-School-Access-Peru/Shape_file/DISTRITOS.shp')
#verifico
print(f"DataFrame cargado con {len(df)} filas")
print(f"Shapefile cargado con {len(peru_distritos)} distritos")

DataFrame cargado con 114919 filas
Shapefile cargado con 1873 distritos


In [8]:
# ----- PARTE 2: PREPARAR LOS DATOS PARA GEOPANDAS -----

# Creo una columna de geometría para convertir el DataFrame a GeoDataFrame
df['geometry'] = df.apply(lambda row: Point(row['Longitud'], row['Latitud']) 
                          if not pd.isna(row['Longitud']) and not pd.isna(row['Latitud']) else None, axis=1)

# Elimino filas sin geometría
df = df.dropna(subset=['geometry'])

# Convierto el DataFrame a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# Me aseguro que el shapefile de distritos tenga el mismo CRS
peru_distritos = peru_distritos.to_crs("EPSG:4326")

In [9]:
# ----- PARTE 3: FILTRAR LOS DATOS POR NIVEL EDUCATIVO -----

# Función para identificar el nivel educativo
def identificar_nivel(nivel_texto):
    if pd.isna(nivel_texto):
        return "Otro"
    
    nivel_texto = str(nivel_texto).lower()
    
    if 'inicial' in nivel_texto:
        return "Inicial"
    elif 'primaria' in nivel_texto:
        return "Primaria"
    elif 'secundaria' in nivel_texto:
        return "Secundaria"
    else:
        return "Otro"
# Aplicamos la función para crear una nueva columna simplificada de nivel
gdf['Nivel_Simple'] = gdf['Nivel / Modalidad'].apply(identificar_nivel)

# Filtro los GeoDataFrames por nivel
gdf_inicial = gdf[gdf['Nivel_Simple'] == "Inicial"]
gdf_primaria = gdf[gdf['Nivel_Simple'] == "Primaria"]
gdf_secundaria = gdf[gdf['Nivel_Simple'] == "Secundaria"]

print(f"Escuelas de nivel Inicial: {len(gdf_inicial)}")
print(f"Escuelas de nivel Primaria: {len(gdf_primaria)}")
print(f"Escuelas de nivel Secundaria: {len(gdf_secundaria)}")

Escuelas de nivel Inicial: 54837
Escuelas de nivel Primaria: 39274
Escuelas de nivel Secundaria: 15789


In [25]:
# ----- PARTE 4: TAREA 1 - CREAR MAPAS ESTÁTICOS POR NIVEL ESCOLAR -----

# Función para contar escuelas por distrito
def contar_escuelas_por_distrito(gdf_escuelas, gdf_distritos):
    # Realizar unión espacial para contar escuelas por distrito
    joined = gpd.sjoin(gdf_escuelas, gdf_distritos, how="inner", predicate="within")
    
    # Cuenta escuelas por distrito
    conteo = joined.groupby(by="DISTRITO").size().reset_index(name="conteo")
    
    # Uno el conteo con el GeoDataFrame de distritos
    resultado = gdf_distritos.merge(conteo, left_on="DISTRITO", right_on="DISTRITO", how="left")
    
    # por las dudas, lleno los valores NaN con 0
    resultado["conteo"] = resultado["conteo"].fillna(0)
    
    return resultado

# Defino una paleta de colores personalizada---que no des calammbre al ojo
colors = ['#ffffcc', '#ffcc00', '#ff6600']  
cmap = LinearSegmentedColormap.from_list("yellow_orange", colors)
    
# Función para crear y guardar el mapa estático
def crear_mapa_estatico(gdf_escuelas, nivel, gdf_distritos):
    # Cuento escuelas por distrito
    distritos_con_conteo = contar_escuelas_por_distrito(gdf_escuelas, gdf_distritos)
    
    # Creo figura
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    
    # Ploteo el mapa
    distritos_con_conteo.plot(column='conteo', cmap=cmap, linewidth=0.5, 
                              ax=ax, edgecolor='0.5', legend=True,
                              legend_kwds={'label': f"Número de escuelas de nivel {nivel} por distrito",
                                           'orientation': "horizontal"})
    # Añado títulos y detalles
    ax.set_title(f'Distribución de escuelas de nivel {nivel} por distrito', fontsize=16)
    ax.set_axis_off()
    # Guardar el mapa
    plt.savefig(f'resultados/mapa_{nivel.lower()}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # para imprimir estadísticas
    total_escuelas = distritos_con_conteo['conteo'].sum()
    distritos_con_escuelas = distritos_con_conteo[distritos_con_conteo['conteo'] > 0].shape[0]
    
    print(f"Nivel {nivel}:")
    print(f"  - Total de escuelas: {total_escuelas}")
    print(f"  - Distritos con al menos una escuela: {distritos_con_escuelas} de {len(gdf_distritos)}")
    print(f"  - Promedio de escuelas por distrito: {total_escuelas / distritos_con_escuelas:.2f}")
    print(f"  - Mapa guardado como: resultados/mapa_{nivel.lower()}.png")
    print()

In [26]:
# Creamos los tres mapas estáticos
print("\n----- TAREA 1: MAPAS ESTÁTICOS POR NIVEL ESCOLAR -----\n")

crear_mapa_estatico(gdf_inicial, "Inicial", peru_distritos)
crear_mapa_estatico(gdf_primaria, "Primaria", peru_distritos)
crear_mapa_estatico(gdf_secundaria, "Secundaria", peru_distritos)


----- TAREA 1: MAPAS ESTÁTICOS POR NIVEL ESCOLAR -----

Nivel Inicial:
  - Total de escuelas: 68720.0
  - Distritos con al menos una escuela: 1871 de 1873
  - Promedio de escuelas por distrito: 36.73
  - Mapa guardado como: resultados/mapa_inicial.png

Nivel Primaria:
  - Total de escuelas: 48651.0
  - Distritos con al menos una escuela: 1872 de 1873
  - Promedio de escuelas por distrito: 25.99
  - Mapa guardado como: resultados/mapa_primaria.png

Nivel Secundaria:
  - Total de escuelas: 19563.0
  - Distritos con al menos una escuela: 1831 de 1873
  - Promedio de escuelas por distrito: 10.68
  - Mapa guardado como: resultados/mapa_secundaria.png



In [29]:
# ----- PARTE 5: TAREA 2 - ANÁLISIS DE PROXIMIDAD -----

print("\n----- TAREA 2: ANÁLISIS DE PROXIMIDAD -----\n")

# Función para realizar análisis de proximidad
def analisis_proximidad(gdf_primaria, gdf_secundaria, region):
    # Filtrar escuelas por región
    gdf_primaria_region = gdf_primaria[gdf_primaria['Departamento'] == region]
    gdf_secundaria_region = gdf_secundaria[gdf_secundaria['Departamento'] == region]
    
    # Filtramos distritos por región
    distritos_region = peru_distritos[peru_distritos['DEPARTAMEN'] == region]
    
    print(f"Análisis para la región {region}:")
    print(f"  - Escuelas primarias: {len(gdf_primaria_region)}")
    print(f"  - Escuelas secundarias: {len(gdf_secundaria_region)}")
    
    # esto lo pongo porsiaca si no hay suficientes escuelas, terminar el análisis
    if len(gdf_primaria_region) < 1 or len(gdf_secundaria_region) < 1:
        print(f"  No hay suficientes escuelas para realizar el análisis de proximidad en {region}.")
        return
    
    # Proyectamos a un CRS que use metros para el análisis de distancia
    gdf_primaria_proj = gdf_primaria_region.to_crs("EPSG:32718")  # UTM zona 18S, que cubre Perú
    gdf_secundaria_proj = gdf_secundaria_region.to_crs("EPSG:32718")
    distritos_region_proj = distritos_region.to_crs("EPSG:32718")
    # Crear un buffer de 5 km para cada escuela primaria
    gdf_primaria_proj['buffer_5km'] = gdf_primaria_proj.geometry.buffer(5000)
    
    # Inicio el contador de escuelas secundarias dentro del radio
    escuelas_cercanas = []
    
    # Para cada escuela primaria, contar cuántas escuelas secundarias están en su radio de 5 km
    for idx, primaria in gdf_primaria_proj.iterrows():
        buffer = primaria['buffer_5km']
        secundarias_dentro = gdf_secundaria_proj[gdf_secundaria_proj.geometry.within(buffer)]
        count = len(secundarias_dentro)
        escuelas_cercanas.append({
            'id_primaria': primaria['Código Modular'],
            'nombre_primaria': primaria['Nombre de SS.EE.'],
            'distrito': primaria['Distrito'],
            'provincia': primaria['Provincia'],
            'latitud': primaria['Latitud'],
            'longitud': primaria['Longitud'],
            'secundarias_cercanas': count,
            'geometry': primaria.geometry,
            'buffer': buffer
        })
    
    # Creo GeoDataFrame con los resultados
    resultados_buffer = gpd.GeoDataFrame(escuelas_cercanas, geometry='geometry', crs="EPSG:32718")
    
    # Encuentro escuela primaria con menos escuelas secundarias cercanas
    min_secundarias = resultados_buffer[resultados_buffer['secundarias_cercanas'] == resultados_buffer['secundarias_cercanas'].min()]
    
    # Encuentro escuela primaria con más escuelas secundarias cercanas
    max_secundarias = resultados_buffer[resultados_buffer['secundarias_cercanas'] == resultados_buffer['secundarias_cercanas'].max()]
    
    #Para imprimir los resultados
    print(f"\nResultados para {region}:")
    print(f"  - Escuela primaria con MENOS escuelas secundarias cercanas:")
    for idx, row in min_secundarias.iterrows():
        print(f"    * {row['nombre_primaria']} (Código: {row['id_primaria']})")
        print(f"      Ubicación: {row['distrito']}, {row['provincia']}")
        print(f"      Coordenadas: {row['latitud']}, {row['longitud']}")
        print(f"      Escuelas secundarias en un radio de 5 km: {row['secundarias_cercanas']}")
    
    print(f"\n  - Escuela primaria con MÁS escuelas secundarias cercanas:")
    for idx, row in max_secundarias.iterrows():
        print(f"    * {row['nombre_primaria']} (Código: {row['id_primaria']})")
        print(f"      Ubicación: {row['distrito']}, {row['provincia']}")
        print(f"      Coordenadas: {row['latitud']}, {row['longitud']}")
        print(f"      Escuelas secundarias en un radio de 5 km: {row['secundarias_cercanas']}")
    
    # Creo y guardo el mapa para la escuela con menos escuelas secundarias cercanas
    for i, row in min_secundarias.head(1).iterrows():
        crear_mapa_proximidad(
            row, 
            gdf_secundaria_proj, 
            distritos_region_proj, 
            region, 
            'min', 
            "Escuela primaria con MENOS escuelas secundarias cercanas"
        )
    
    # Creo y guardo el mapa para la escuela con más escuelas secundarias cercanas
    for i, row in max_secundarias.head(1).iterrows():
        crear_mapa_proximidad(
            row, 
            gdf_secundaria_proj, 
            distritos_region_proj, 
            region, 
            'max', 
            "Escuela primaria con MÁS escuelas secundarias cercanas"
        )


----- TAREA 2: ANÁLISIS DE PROXIMIDAD -----



In [30]:
# Función para crear el mapa de proximidad
def crear_mapa_proximidad(escuela_primaria, gdf_secundaria, gdf_distritos, region, tipo, titulo):
    # Obtener escuelas secundarias dentro del buffer
    secundarias_dentro = gdf_secundaria[gdf_secundaria.geometry.within(escuela_primaria['buffer'])]
    
    # Creo figura
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    
    # Ploteo el mapa base (distritos)
    gdf_distritos.plot(ax=ax, color='lightgray', edgecolor='darkgray', alpha=0.5, linewidth=1)
    
    # Ploteo el buffer de 5 km
    gpd.GeoSeries([escuela_primaria['buffer']], crs=gdf_secundaria.crs).plot(
        ax=ax, color='blue', alpha=0.2, linewidth=1
    )
    
    # Ploteo las escuelas secundarias dentro del buffer
    secundarias_dentro.plot(ax=ax, color='green', marker='o', markersize=50, label='Escuelas Secundarias')
    
    # Ploteo la escuela primaria central
    gpd.GeoSeries([escuela_primaria.geometry], crs=gdf_secundaria.crs).plot(
        ax=ax, color='red', marker='*', markersize=150, label='Escuela Primaria'
    )
    
    # Añado títulos y leyenda
    ax.set_title(f"{titulo}\n{escuela_primaria['nombre_primaria']} - {region}", fontsize=14)
    ax.set_xlabel('Longitud')
    ax.set_ylabel('Latitud')
    plt.legend()
    
    # Guardo el mapa
    plt.savefig(f'resultados/proximidad_{region}_{tipo}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  - Mapa guardado como: resultados/proximidad_{region}_{tipo}.png")

# Realizamos el análisis de proximidad para las regiones especificadas
analisis_proximidad(gdf_primaria, gdf_secundaria, "HUANCAVELICA")
analisis_proximidad(gdf_primaria, gdf_secundaria, "AYACUCHO")

print("\nAnálisis completado. Todos los resultados han sido guardados en la carpeta 'resultados'.")

Análisis para la región HUANCAVELICA:
  - Escuelas primarias: 1224
  - Escuelas secundarias: 397

Resultados para HUANCAVELICA:
  - Escuela primaria con MENOS escuelas secundarias cercanas:
    * 36062 (Código: 428391)
      Ubicación: HUANCAVELICA, HUANCAVELICA
      Coordenadas: -12.8988, -74.93421
      Escuelas secundarias en un radio de 5 km: 0
    * 36342 (Código: 420042)
      Ubicación: ACOBAMBILLA, HUANCAVELICA
      Coordenadas: -12.71392, -75.43308
      Escuelas secundarias en un radio de 5 km: 0
    * 36035 (Código: 1232073)
      Ubicación: ACOBAMBILLA, HUANCAVELICA
      Coordenadas: -12.75668, -75.30507
      Escuelas secundarias en un radio de 5 km: 0
    * 36308 (Código: 428441)
      Ubicación: HUACHOCOLPA, HUANCAVELICA
      Coordenadas: -13.12433, -74.95256
      Escuelas secundarias en un radio de 5 km: 0
    * 37504 (Código: 421149)
      Ubicación: HUACHOCOLPA, HUANCAVELICA
      Coordenadas: -13.06925, -74.98149
      Escuelas secundarias en un radio de 5 km: 0

C:\Users\Nadia Copello\AppData\Local\Temp\ipykernel_7684\1570238510.py:18: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  secundarias_dentro.plot(ax=ax, color='green', marker='o', markersize=50, label='Escuelas Secundarias')


  - Mapa guardado como: resultados/proximidad_HUANCAVELICA_min.png
  - Mapa guardado como: resultados/proximidad_HUANCAVELICA_max.png
Análisis para la región AYACUCHO:
  - Escuelas primarias: 1435
  - Escuelas secundarias: 542

Resultados para AYACUCHO:
  - Escuela primaria con MENOS escuelas secundarias cercanas:
    * 38783 (Código: 556779)
      Ubicación: CHIARA, HUAMANGA
      Coordenadas: -13.2953, -74.1426
      Escuelas secundarias en un radio de 5 km: 0
    * 38089 (Código: 442194)
      Ubicación: SANTIAGO DE PISCHA, HUAMANGA
      Coordenadas: -13.0297, -74.3781
      Escuelas secundarias en un radio de 5 km: 0
    * 38978 (Código: 769604)
      Ubicación: SOCOS, HUAMANGA
      Coordenadas: -13.28787745, -74.26761624
      Escuelas secundarias en un radio de 5 km: 0
    * 38910 (Código: 670380)
      Ubicación: VINCHOS, HUAMANGA
      Coordenadas: -13.3122, -74.5015
      Escuelas secundarias en un radio de 5 km: 0
    * 38935 (Código: 670620)
      Ubicación: VINCHOS, HUAMA

C:\Users\Nadia Copello\AppData\Local\Temp\ipykernel_7684\1570238510.py:18: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  secundarias_dentro.plot(ax=ax, color='green', marker='o', markersize=50, label='Escuelas Secundarias')


  - Mapa guardado como: resultados/proximidad_AYACUCHO_min.png
  - Mapa guardado como: resultados/proximidad_AYACUCHO_max.png

Análisis completado. Todos los resultados han sido guardados en la carpeta 'resultados'.


In [ ]:
####AHORA VAMOS A HACER LO DINÁMICO

In [ ]:
####

In [ ]:
####

In [ ]:
####

In [31]:
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth, LayerControl, Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import numpy as np
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import branca.colormap as cm
import os
from folium.features import DivIcon

In [32]:
# ----- PARTE 1: CARGAR Y PREPARAR LOS DATOS -----

# Cargo el DataFrame de instituciones educativas
archivo_html = 'listado_iiee.xls'
df = pd.read_html(archivo_html)[0]

# Cargo el shapefile de distritos
peru_distritos = gpd.read_file('C:/Users/Nadia Copello/Documents/GitHub/High-School-Access-Peru/Shape_file/DISTRITOS.shp')
#imprimo para verificar
print(f"DataFrame cargado con {len(df)} filas")
print(f"Shapefile cargado con {len(peru_distritos)} distritos")

DataFrame cargado con 114919 filas
Shapefile cargado con 1873 distritos


In [33]:
# ----- PARTE 2: PREPARAR LOS DATOS PARA GEOPANDAS -----

# Creo una columna de geometría para convertir el DataFrame a GeoDataFrame
df['geometry'] = df.apply(lambda row: Point(row['Longitud'], row['Latitud']) 
                          if not pd.isna(row['Longitud']) and not pd.isna(row['Latitud']) else None, axis=1)

# Elimino filas sin geometría
df = df.dropna(subset=['geometry'])

# Convierto el DataFrame a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# Me aseguro que el shapefile de distritos tenga el mismo CRS
peru_distritos = peru_distritos.to_crs("EPSG:4326")

In [34]:
#----- PARTE 3: FILTRAR LOS DATOS POR NIVEL EDUCATIVO -----

# Función para identificar el nivel educativo
def identificar_nivel(nivel_texto):
    if pd.isna(nivel_texto):
        return "Otro"
    
    nivel_texto = str(nivel_texto).lower()
    
    if 'inicial' in nivel_texto:
        return "Inicial"
    elif 'primaria' in nivel_texto:
        return "Primaria"
    elif 'secundaria' in nivel_texto:
        return "Secundaria"
    else:
        return "Otro"
# Aplico la función para crear una nueva columna simplificada de nivel
gdf['Nivel_Simple'] = gdf['Nivel / Modalidad'].apply(identificar_nivel)

# Filtro los GeoDataFrames por nivel
gdf_inicial = gdf[gdf['Nivel_Simple'] == "Inicial"]
gdf_primaria = gdf[gdf['Nivel_Simple'] == "Primaria"]
gdf_secundaria = gdf[gdf['Nivel_Simple'] == "Secundaria"]

print(f"Escuelas de nivel Inicial: {len(gdf_inicial)}")
print(f"Escuelas de nivel Primaria: {len(gdf_primaria)}")
print(f"Escuelas de nivel Secundaria: {len(gdf_secundaria)}")

Escuelas de nivel Inicial: 54837
Escuelas de nivel Primaria: 39274
Escuelas de nivel Secundaria: 15789


In [44]:
import branca.colormap as cm
from branca.colormap import linear

In [49]:
# ----- PARTE 4: TAREA 1 - CREAR MAPA COROPLÉTICO CON FOLIUM -----

# Función para contar escuelas por distrito
def contar_escuelas_por_distrito(gdf_escuelas, gdf_distritos):
    # Realizo unión espacial para contar escuelas por distrito
    joined = gpd.sjoin(gdf_escuelas, gdf_distritos, how="inner", predicate="within")
    
    # Cuento escuelas por distrito
    conteo = joined.groupby(by="DISTRITO").size().reset_index(name="conteo")
    
    # Uno el conteo con el GeoDataFrame de distritos
    resultado = gdf_distritos.merge(conteo, left_on="DISTRITO", right_on="DISTRITO", how="left")
    
    # Lleno por siaca los valores NaN con 0
    resultado["conteo"] = resultado["conteo"].fillna(0)
    
    return resultado

# Para obtener conteos por nivel educativo
distritos_inicial = contar_escuelas_por_distrito(gdf_inicial, peru_distritos)
distritos_primaria = contar_escuelas_por_distrito(gdf_primaria, peru_distritos)
distritos_secundaria = contar_escuelas_por_distrito(gdf_secundaria, peru_distritos)


In [50]:
# Crear un mapa coroplético con Folium
def crear_mapa_coropletico_folium():
    # Creo un mapa base centrado en Perú
    m = folium.Map(location=[-9.1900, -75.0152], zoom_start=6, tiles='OpenStreetMap')
    
    # Añado capa de escuelas iniciales
    escala_inicial = cm.linear.YlOrRd_09.scale(
        distritos_inicial['conteo'].min(),
        distritos_inicial['conteo'].max()
    )
    
    estilo_inicial = lambda x: {
        'fillColor': escala_inicial(x['properties']['conteo']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    }
    
    geojson_inicial = folium.GeoJson(
        distritos_inicial.to_json(),
        name='Inicial',
        style_function=estilo_inicial,
        tooltip=folium.GeoJsonTooltip(
            fields=['DISTRITO', 'conteo'],
            aliases=['Distrito', 'Escuelas Iniciales'],
            localize=True
        )
    )
    
    # Añado capa de escuelas primarias
    escala_primaria = escala_primaria = cm.linear.OrRd_09.scale(
        distritos_primaria['conteo'].min(),
        distritos_primaria['conteo'].max()
    )
    
    estilo_primaria = lambda x: {
        'fillColor': escala_primaria(x['properties']['conteo']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    }
    
    geojson_primaria = folium.GeoJson(
        distritos_primaria.to_json(),
        name='Primaria',
        style_function=estilo_primaria,
        tooltip=folium.GeoJsonTooltip(
            fields=['DISTRITO', 'conteo'],
            aliases=['Distrito', 'Escuelas Primarias'],
            localize=True
        )
    )
    
    # Añado capa de escuelas secundarias
    escala_secundaria = cm.linear.Blues_09.scale(
        distritos_secundaria['conteo'].min(),
        distritos_secundaria['conteo'].max()
    )
    
    estilo_secundaria = lambda x: {
        'fillColor': escala_secundaria(x['properties']['conteo']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    }
    
    geojson_secundaria = folium.GeoJson(
        distritos_secundaria.to_json(),
        name='Secundaria',
        style_function=estilo_secundaria,
        tooltip=folium.GeoJsonTooltip(
            fields=['DISTRITO', 'conteo'],
            aliases=['Distrito', 'Escuelas Secundarias'],
            localize=True
        )
    )
    
    # Añado leyendas para cada capa
    escala_inicial.caption = 'Número de escuelas iniciales'
    escala_primaria.caption = 'Número de escuelas primarias'
    escala_secundaria.caption = 'Número de escuelas secundarias'
    
    # Añado todas las capas al mapa
    geojson_inicial.add_to(m)
    m.add_child(escala_inicial)
    
    geojson_primaria.add_to(m)
    m.add_child(escala_primaria)
    
    geojson_secundaria.add_to(m)
    m.add_child(escala_secundaria)
    
    # Añado control de capas
    folium.LayerControl().add_to(m)
    
    # Guardo el mapa
    m.save('mapa_escuelas_peru_coropletico.html')
    
    print("Mapa coroplético creado y guardado como 'mapa_escuelas_peru_coropletico.html'")
    return m

# Crear el mapa coroplético para Perú
mapa_coropletico = crear_mapa_coropletico_folium()

Mapa coroplético creado y guardado como 'mapa_escuelas_peru_coropletico.html'


In [ ]:
####parte 2######
###################para crear lo de proximidad
####################

In [61]:
#import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

# Función para crear mapa de proximidad para una región específica
def crear_mapa_proximidad_folium(region):
    # Filtrar datos para la región
    gdf_primaria_region = gdf_primaria[gdf_primaria['Departamento'] == region].copy()
    gdf_secundaria_region = gdf_secundaria[gdf_secundaria['Departamento'] == region].copy()
    distritos_region = peru_distritos[peru_distritos['DEPARTAMEN'] == region].copy()
    
    # Verifico si hay datos suficientes
    if len(gdf_primaria_region) == 0 or len(gdf_secundaria_region) == 0:
        print(f"No hay suficientes datos para la región {region}")
        return None
    
    # Proyecto a un CRS que use metros para el análisis de distancia
    gdf_primaria_proj = gdf_primaria_region.to_crs("EPSG:32718")  # UTM zona 18S para Perú
    gdf_secundaria_proj = gdf_secundaria_region.to_crs("EPSG:32718")
    
    # Buffer de 5 km
    gdf_primaria_proj['buffer_5km'] = gdf_primaria_proj.geometry.buffer(5000)
    
    # Para volver a proyectar a WGS84 para Folium
    gdf_primaria_region['buffer_5km'] = gdf_primaria_proj['buffer_5km'].to_crs("EPSG:4326")
    
    # Cuento escuelas secundarias en el radio de 5 km
    escuelas_cercanas = []
    for idx, primaria in gdf_primaria_region.iterrows():
        buffer = primaria['buffer_5km']
        secundarias_dentro = gdf_secundaria_region[gdf_secundaria_region.geometry.within(buffer)]
        count = len(secundarias_dentro)
        
        escuelas_cercanas.append({
            'id_primaria': primaria['Código Modular'],
            'nombre_primaria': primaria['Nombre de SS.EE.'],
            'distrito': primaria['Distrito'],
            'provincia': primaria['Provincia'],
            'latitud': primaria['Latitud'],
            'longitud': primaria['Longitud'],
            'secundarias_cercanas': count,
            'geometry': primaria.geometry,
            'buffer': buffer
        })
    
    resultados_gdf = gpd.GeoDataFrame(escuelas_cercanas, geometry='geometry', crs="EPSG:4326")
    
    # para escuelas con menos secundarias (al menos 1, si existe)
    resultados_con_secundarias = resultados_gdf[resultados_gdf['secundarias_cercanas'] > 0]
    if len(resultados_con_secundarias) > 0:
        min_secundarias = resultados_con_secundarias.loc[resultados_con_secundarias['secundarias_cercanas'].idxmin()]
    else:
        min_secundarias = resultados_gdf.loc[resultados_gdf['secundarias_cercanas'].idxmin()]
    
    max_secundarias = resultados_gdf.loc[resultados_gdf['secundarias_cercanas'].idxmax()]
    
    # Creo mapa
    m = folium.Map(location=[gdf_primaria_region['Latitud'].mean(), gdf_primaria_region['Longitud'].mean()],
                   zoom_start=9,
                   tiles='OpenStreetMap')
    
    # Añado distritos
    folium.GeoJson(
        distritos_region,
        name='Distritos',
        style_function=lambda x: {
            'fillColor': 'lightgray',
            'color': 'gray',
            'weight': 1,
            'fillOpacity': 0.3
        }
    ).add_to(m)
    
    # Para los Clusters
    cluster_primarias = MarkerCluster(name='Escuelas Primarias').add_to(m)
    cluster_secundarias = MarkerCluster(name='Escuelas Secundarias').add_to(m)
    
    # Escuelas primarias
    for idx, row in gdf_primaria_region.iterrows():
        sec_cercanas = resultados_gdf[resultados_gdf['id_primaria'] == row['Código Modular']]['secundarias_cercanas'].values[0]
        popup_text = f"""
        <b>{row['Nombre de SS.EE.']}</b><br>
        Distrito: {row['Distrito']}<br>
        Provincia: {row['Provincia']}<br>
        Escuelas secundarias cercanas: {sec_cercanas}
        """
        folium.Marker(
            location=[row['Latitud'], row['Longitud']],
            popup=popup_text,
            tooltip=row['Nombre de SS.EE.'],
            icon=folium.Icon(color='blue', icon='school', prefix='fa')
        ).add_to(cluster_primarias)
    
    # Escuelas secundarias
    for idx, row in gdf_secundaria_region.iterrows():
        popup_text = f"""
        <b>{row['Nombre de SS.EE.']}</b><br>
        Distrito: {row['Distrito']}<br>
        Provincia: {row['Provincia']}
        """
        folium.Marker(
            location=[row['Latitud'], row['Longitud']],
            popup=popup_text,
            tooltip=row['Nombre de SS.EE.'],
            icon=folium.Icon(color='green', icon='book', prefix='fa')
        ).add_to(cluster_secundarias)
    
    # Escuela con menos secundarias cercanas
    folium.Marker(
        location=[min_secundarias['latitud'], min_secundarias['longitud']],
        popup=f"""
        <b>Escuela con MENOS secundarias cercanas</b><br>
        Nombre: {min_secundarias['nombre_primaria']}<br>
        Distrito: {min_secundarias['distrito']}<br>
        Provincia: {min_secundarias['provincia']}<br>
        Secundarias en 5 km: {min_secundarias['secundarias_cercanas']}
        """,
        tooltip=f"MENOS secundarias: {min_secundarias['secundarias_cercanas']}",
        icon=folium.Icon(color='red', icon='exclamation-circle', prefix='fa')
    ).add_to(m)

    # Buffer menos
    folium.GeoJson(
        gpd.GeoDataFrame(geometry=[min_secundarias['buffer']], crs="EPSG:4326").to_json(),
        name="Radio 5 km (MENOS)",
        style_function=lambda x: {
            'fillColor': 'red',
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0.1
        }
    ).add_to(m)
    
    # Escuela con más secundarias cercanas
    folium.Marker(
        location=[max_secundarias['latitud'], max_secundarias['longitud']],
        popup=f"""
        <b>Escuela con MÁS secundarias cercanas</b><br>
        Nombre: {max_secundarias['nombre_primaria']}<br>
        Distrito: {max_secundarias['distrito']}<br>
        Provincia: {max_secundarias['provincia']}<br>
        Secundarias en 5 km: {max_secundarias['secundarias_cercanas']}
        """,
        tooltip=f"MÁS secundarias: {max_secundarias['secundarias_cercanas']}",
        icon=folium.Icon(color='darkgreen', icon='check-circle', prefix='fa')
    ).add_to(m)

    # Buffer más
    folium.GeoJson(
        gpd.GeoDataFrame(geometry=[max_secundarias['buffer']], crs="EPSG:4326").to_json(),
        name="Radio 5 km (MÁS)",
        style_function=lambda x: {
            'fillColor': 'green',
            'color': 'green',
            'weight': 2,
            'fillOpacity': 0.1
        }
    ).add_to(m)

    # Secundarias dentro del buffer máximo
    secundarias_dentro_max = gdf_secundaria_region[gdf_secundaria_region.geometry.within(max_secundarias['buffer'])]
    for idx, row in secundarias_dentro_max.iterrows():
        popup_text = f"""
        <b>{row['Nombre de SS.EE.']}</b><br>
        Dentro del radio de 5 km de la escuela con MÁS secundarias
        """
        folium.Marker(
            location=[row['Latitud'], row['Longitud']],
            popup=popup_text,
            icon=folium.Icon(color='darkgreen', icon='graduation-cap', prefix='fa')
        ).add_to(m)

    # Secundarias dentro del buffer mínimo
    secundarias_dentro_min = gdf_secundaria_region[gdf_secundaria_region.geometry.within(min_secundarias['buffer'])]
    for idx, row in secundarias_dentro_min.iterrows():
        popup_text = f"""
        <b>{row['Nombre de SS.EE.']}</b><br>
        Dentro del radio de 5 km de la escuela con MENOS secundarias
        """
        folium.Marker(
            location=[row['Latitud'], row['Longitud']],
            popup=popup_text,
            icon=folium.Icon(color='orange', icon='graduation-cap', prefix='fa')
        ).add_to(m)
    
    # Control de capas
    folium.LayerControl().add_to(m)

    # Título del mapa
    title_html = f'''
    <h3 align="center" style="font-size:16px"><b>Análisis de Proximidad de Escuelas en {region}</b></h3>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    return {
    "region": region,
    "max_secundarias": max_secundarias,
    "min_secundarias": min_secundarias
        
}, m
resultado_hvca, mapa_hvca = tu_funcion("HUANCAVELICA")
print(resultado_hvca['region'])        # ✅ Acceso correcto
mapa_hvca                              # ✅ Muestra el mapa


NameError: name 'tu_funcion' is not defined

In [ ]:
####para hacer el streamlit

In [ ]:
#####################################

In [ ]:
######################################

In [92]:
df = pd.read_html(archivo_html)[0]
distritos = gpd.read_file('C:/Users/Nadia Copello/Documents/GitHub/High-School-Access-Peru/Shape_file/DISTRITOS.shp')

# Filtrar por nivel educativo
df_primaria = df[df['Nivel / Modalidad'] == 'Primaria']
df_secundaria = df[df['Nivel / Modalidad'] == 'Secundaria']

# Para primaria
gdf_primaria = gpd.GeoDataFrame(
    df_primaria,
    geometry=gpd.points_from_xy(df_primaria['Longitud'], df_primaria['Latitud']),
    crs='EPSG:4326'
)

# Para secundaria
gdf_secundaria = gpd.GeoDataFrame(
    df_secundaria,
    geometry=gpd.points_from_xy(df_secundaria['Longitud'], df_secundaria['Latitud']),
    crs='EPSG:4326'
)
gdf_primaria.to_file("escuelas_primaria.shp")
gdf_secundaria.to_file("escuelas_secundaria.shp")

C:\Users\Nadia Copello\AppData\Local\Temp\ipykernel_7684\1651363098.py:21: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_primaria.to_file("escuelas_primaria.shp")
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Código Modular' to 'Código Mo'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Nombre de SS.EE.' to 'Nombre de'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Departamento' to 'Departamen'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Código DRE/UGEL' to 'Código DR'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field n

In [129]:
# Cargar datasets
primarias = gdf_primaria.to_file("escuelas_primaria.shp")
secundarias = gdf_secundaria.to_file("escuelas_secundaria.shp")

C:\Users\Nadia Copello\AppData\Local\Temp\ipykernel_7684\3437182816.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  primarias = gdf_primaria.to_file("escuelas_primaria.shp")
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Código Modular' to 'Código Mo'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Nombre de SS.EE.' to 'Nombre de'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Departamento' to 'Departamen'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Código DRE/UGEL' to 'Código DR'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/launde

In [130]:
primarias = gpd.read_file("C:/Users/Nadia Copello/Documents/GitHub/High-School-Access-Peru/escuelas_primaria.shp")
print(primarias.head())

   Código Mo  Anexo                    Nombre de  Ubigeo Departamen  \
0     257329      0           18001 MIGUEL RUBIO   10101   AMAZONAS   
1     257337      0      18002 MARIA AUXILIADORA   10101   AMAZONAS   
2     257345      0             18003 SANTA ROSA   10101   AMAZONAS   
3    1311109      0                        18006   10101   AMAZONAS   
4     255919      0  18288 ISABEL LINCH DE RUBIO   10101   AMAZONAS   

     Provincia     Distrito  Código DR   DRE / UGEL   Centro Pob  Código Ce  \
0  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
1  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
2  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
3  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
4  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   

   Código Lo                  Dirección Nivel / Mo  \
0      316.0        JIRON LA MERCED 859   Pr

In [131]:
secundarias = gpd.read_file("C:/Users/Nadia Copello/Documents/GitHub/High-School-Access-Peru/escuelas_secundaria.shp")
print(secundarias.head())

   Código Mo  Anexo                    Nombre de  Ubigeo Departamen  \
0     643668      0           18001 MIGUEL RUBIO   10101   AMAZONAS   
1    1302017      0      18002 MARIA AUXILIADORA   10101   AMAZONAS   
2    3024064      0  18288 ISABEL LINCH DE RUBIO   10101   AMAZONAS   
3    1302132      0                 ALFRED NOBEL   10101   AMAZONAS   
4    1582550      0              DIVINO SALVADOR   10101   AMAZONAS   

     Provincia     Distrito  Código DR   DRE / UGEL   Centro Pob  Código Ce  \
0  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
1  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
2  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
3  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
4  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   

   Código Lo            Dirección  Nivel / Mo                   Gestion /  \
0      316.0  JIRON L

In [132]:
print(primarias.columns)
print(secundarias.columns)

Index(['Código Mo', 'Anexo', 'Nombre de', 'Ubigeo', 'Departamen', 'Provincia',
       'Distrito', 'Código DR', 'DRE / UGEL', 'Centro Pob', 'Código Ce',
       'Código Lo', 'Dirección', 'Nivel / Mo', 'Gestion /', 'Latitud',
       'Longitud', 'Altitud', 'Fuente de', 'geometry'],
      dtype='object')
Index(['Código Mo', 'Anexo', 'Nombre de', 'Ubigeo', 'Departamen', 'Provincia',
       'Distrito', 'Código DR', 'DRE / UGEL', 'Centro Pob', 'Código Ce',
       'Código Lo', 'Dirección', 'Nivel / Mo', 'Gestion /', 'Latitud',
       'Longitud', 'Altitud', 'Fuente de', 'geometry'],
      dtype='object')


In [133]:

# Filtrar solo Huancavelica y Ayacucho
deps = ["HUANCAVELICA"]
primarias = primarias[primarias["Departamen"].isin(deps)]
secundarias = secundarias[secundarias["Departamen"].isin(deps)]


# Asegúrate de que ambas estén en el mismo CRS (usa WGS84 para Folium)
primarias = primarias.to_crs(epsg=4326)
secundarias = secundarias.to_crs(epsg=4326)

# Convierte a CRS métrico (para medir distancia en metros)
primarias_m = primarias.to_crs(epsg=32718)  # UTM zona 18S (Perú)
secundarias_m = secundarias.to_crs(epsg=32718)

# Calcular cuántas secundarias están a menos de 5km de cada primaria
def contar_secundarias_cercanas(escuela, secundarias_gdf, radio=5000):
    buffer = escuela.geometry.buffer(radio)
    return secundarias_gdf[secundarias_gdf.geometry.within(buffer)].shape[0]

primarias_m["secundarias_cercanas"] = primarias_m.apply(
    lambda row: contar_secundarias_cercanas(row, secundarias_m), axis=1
)

# Obtener la escuela con más secundarias cercanas
mejor_primaria = primarias_m.sort_values("secundarias_cercanas", ascending=False).iloc[0]
import folium

# Volver a WGS84 para visualización
primaria_wgs = gpd.GeoDataFrame(geometry=[mejor_primaria.geometry], crs="EPSG:32718").to_crs(epsg=4326)
secundarias_wgs = secundarias_m.to_crs(epsg=4326)

# Crear mapa
centro = primaria_wgs.geometry.iloc[0].coords[0][::-1]
m = folium.Map(location=centro, zoom_start=12)

# Agregar primaria
folium.Marker(location=centro, popup="Primaria centroide", icon=folium.Icon(color="blue")).add_to(m)

# Dibujar círculo de 5km
folium.Circle(
    location=centro,
    radius=5000,
    color="green",
    fill=True,
    fill_opacity=0.1
).add_to(m)

# Agregar secundarias dentro del radio
for _, row in secundarias_wgs.iterrows():
    dist = row.geometry.distance(primaria_wgs.geometry.iloc[0])
    if dist <= 0.045:  # ~5 km en grados
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            icon=folium.Icon(color="red"),
            popup="Secundaria"
        ).add_to(m)

m.save("mapa_proximidad1.html")





In [134]:
import webbrowser
webbrowser.open("mapa_proximidad1.html")

True

In [ ]:
##################AHORA CON AYACUCHO###########################

In [141]:
# Cargar datasets
primarias = gdf_primaria.to_file("escuelas_primaria.shp")
secundarias = gdf_secundaria.to_file("escuelas_secundaria.shp")

C:\Users\Nadia Copello\AppData\Local\Temp\ipykernel_7684\3437182816.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  primarias = gdf_primaria.to_file("escuelas_primaria.shp")
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Código Modular' to 'Código Mo'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Nombre de SS.EE.' to 'Nombre de'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Departamento' to 'Departamen'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Código DRE/UGEL' to 'Código DR'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/launde

In [142]:
primarias = gpd.read_file("C:/Users/Nadia Copello/Documents/GitHub/High-School-Access-Peru/escuelas_primaria.shp")
print(primarias.head())

   Código Mo  Anexo                    Nombre de  Ubigeo Departamen  \
0     257329      0           18001 MIGUEL RUBIO   10101   AMAZONAS   
1     257337      0      18002 MARIA AUXILIADORA   10101   AMAZONAS   
2     257345      0             18003 SANTA ROSA   10101   AMAZONAS   
3    1311109      0                        18006   10101   AMAZONAS   
4     255919      0  18288 ISABEL LINCH DE RUBIO   10101   AMAZONAS   

     Provincia     Distrito  Código DR   DRE / UGEL   Centro Pob  Código Ce  \
0  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
1  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
2  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
3  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
4  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   

   Código Lo                  Dirección Nivel / Mo  \
0      316.0        JIRON LA MERCED 859   Pr

In [143]:
secundarias = gpd.read_file("C:/Users/Nadia Copello/Documents/GitHub/High-School-Access-Peru/escuelas_secundaria.shp")
print(secundarias.head())

   Código Mo  Anexo                    Nombre de  Ubigeo Departamen  \
0     643668      0           18001 MIGUEL RUBIO   10101   AMAZONAS   
1    1302017      0      18002 MARIA AUXILIADORA   10101   AMAZONAS   
2    3024064      0  18288 ISABEL LINCH DE RUBIO   10101   AMAZONAS   
3    1302132      0                 ALFRED NOBEL   10101   AMAZONAS   
4    1582550      0              DIVINO SALVADOR   10101   AMAZONAS   

     Provincia     Distrito  Código DR   DRE / UGEL   Centro Pob  Código Ce  \
0  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
1  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
2  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
3  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   
4  CHACHAPOYAS  CHACHAPOYAS      10001  CHACHAPOYAS  CHACHAPOYAS   131197.0   

   Código Lo            Dirección  Nivel / Mo                   Gestion /  \
0      316.0  JIRON L

In [144]:
print(primarias.columns)
print(secundarias.columns)

Index(['Código Mo', 'Anexo', 'Nombre de', 'Ubigeo', 'Departamen', 'Provincia',
       'Distrito', 'Código DR', 'DRE / UGEL', 'Centro Pob', 'Código Ce',
       'Código Lo', 'Dirección', 'Nivel / Mo', 'Gestion /', 'Latitud',
       'Longitud', 'Altitud', 'Fuente de', 'geometry'],
      dtype='object')
Index(['Código Mo', 'Anexo', 'Nombre de', 'Ubigeo', 'Departamen', 'Provincia',
       'Distrito', 'Código DR', 'DRE / UGEL', 'Centro Pob', 'Código Ce',
       'Código Lo', 'Dirección', 'Nivel / Mo', 'Gestion /', 'Latitud',
       'Longitud', 'Altitud', 'Fuente de', 'geometry'],
      dtype='object')


In [145]:
# Filtrar solo Huancavelica y Ayacucho
deps = ["AYACUCHO"]
primarias = primarias[primarias["Departamen"].isin(deps)]
secundarias = secundarias[secundarias["Departamen"].isin(deps)]


# Asegúrate de que ambas estén en el mismo CRS (usa WGS84 para Folium)
primarias = primarias.to_crs(epsg=4326)
secundarias = secundarias.to_crs(epsg=4326)

# Convierte a CRS métrico (para medir distancia en metros)
primarias_m = primarias.to_crs(epsg=32718)  # UTM zona 18S (Perú)
secundarias_m = secundarias.to_crs(epsg=32718)

# Calcular cuántas secundarias están a menos de 5km de cada primaria
def contar_secundarias_cercanas(escuela, secundarias_gdf, radio=5000):
    buffer = escuela.geometry.buffer(radio)
    return secundarias_gdf[secundarias_gdf.geometry.within(buffer)].shape[0]

primarias_m["secundarias_cercanas"] = primarias_m.apply(
    lambda row: contar_secundarias_cercanas(row, secundarias_m), axis=1
)

# Obtener la escuela con más secundarias cercanas
mejor_primaria = primarias_m.sort_values("secundarias_cercanas", ascending=False).iloc[0]
import folium
# Volver a WGS84 para visualización
primaria_wgs = gpd.GeoDataFrame(geometry=[mejor_primaria.geometry], crs="EPSG:32718").to_crs(epsg=4326)
secundarias_wgs = secundarias_m.to_crs(epsg=4326)

# Crear mapa
centro = primaria_wgs.geometry.iloc[0].coords[0][::-1]
m = folium.Map(location=centro, zoom_start=12)

# Agregar primaria
folium.Marker(location=centro, popup="Primaria centroide", icon=folium.Icon(color="blue")).add_to(m)

# Dibujar círculo de 5km
folium.Circle(
    location=centro,
    radius=5000,
    color="green",
    fill=True,
    fill_opacity=0.1
).add_to(m)

# Agregar secundarias dentro del radio
for _, row in secundarias_wgs.iterrows():
    dist = row.geometry.distance(primaria_wgs.geometry.iloc[0])
    if dist <= 0.045:  # ~5 km en grados
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            icon=folium.Icon(color="red"),
            popup="Secundaria"
        ).add_to(m)

m.save("mapa_proximidad2.html")

In [146]:
import webbrowser
webbrowser.open("mapa_proximidad2.html")

True

In [ ]:
######AHORA EL STREAMLIST#### :ccc

In [64]:
pip install streamlit-folium

In [84]:
import streamlit as st
import geopandas as gpd
import folium
from streamlit_folium import folium_static
import matplotlib.pyplot as plt


# ----------- PESTAÑAS -------------
tab1, tab2, tab3 = st.tabs(["📊 Descripción de los datos", "🗺️ Mapas estáticos", "🌍 Mapas dinámicos"])

# ----------- PESTAÑA 1 -------------
with tab1:
    st.header("📊 Descripción de los datos")
    
    st.markdown("""
    **Unidad de análisis:** Escuelas primarias y secundarias en determinada región del Perú.  
    **Fuentes de datos:** Bases del MINEDU (padron de II.EE) descargadas en formato CSV o SHP.  
    **Supuestos o preprocesamiento:**  
    - Se transformaron los datos a GeoDataFrames.  
    - Se crearon buffers de 5 km para analizar cercanía entre primarias y secundarias.
    """)

# ----------- PESTAÑA 2 -------------
with tab2:
    st.header("🗺️ Mapas estáticos")

    # Carga o genera un mapa estático usando GeoPandas y matplotlib
    gdf_primaria = gpd.read_file("escuelas_primaria.shp")
    gdf_secundaria = gpd.read_file("escuelas_secundaria.shp")

    fig, ax = plt.subplots()
    gdf_primaria.plot(ax=ax, color='blue', label='Primarias')
    gdf_secundaria.plot(ax=ax, color='orange', label='Secundarias')
    plt.legend()
    st.pyplot(fig)

# ----------- PESTAÑA 3 -------------
with tab3:
    st.header("🌍 Mapas dinámicos")

    # Centrar el mapa en la región de análisis
    m = folium.Map(location=[-12.0, -76.9], zoom_start=8)

    folium_static(m)


C:\Users\Nadia Copello\AppData\Local\Temp\ipykernel_7684\2585925970.py:26: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_primaria.to_file("escuelas_primaria.shp")
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Código Modular' to 'Código Mo'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Nombre de SS.EE.' to 'Nombre de'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Departamento' to 'Departamen'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Código DRE/UGEL' to 'Código DR'
  ogr_write(
C:\Users\Nadia Copello\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field n